## Dynamics

In psim, the coordinates are just rectangular. We define the coordinates as follows:
- $\vec r^{ECI} = (r_1, r_2, r_3)$ is the position of the spacecraft COM in ECI.
- The body frame of the spacecraft is defined with:
  - The $x$-axis pointing from mass A ($m_A$) to mass B ($m_B$).
  - We arbitrarily give at least one of the masses a small rectangular shape. We have the rectangle share a plane with the linkage between the two masses, and then define the surface normal to the rectangle that rotates $A$ to $B$ CCW as the $y$-axis of the body frame.
  - The $z$-axis that completes the right-handed triad.

$^{B}q^{ECI}$, or simply $q$, transforms ECI-frame vectors to body frame vectors. We'll denote the conjugate of this quaternion as $\overline q$.

The Lagrangian is given by the translational kinetic energy of the spacecraft CM, the rotational kinetic energy about the CM, and the gravitational potential energy of the system:

$$\mathcal L = T - V = KE_{trans} + KE_{rot} - (- m_A G(\vec r_A^{ECI}) - m_B G(\vec r_B^{ECI}))$$

Here, $\vec r_A^{ECI}$ and $\vec r_B^{ECI}$ are the vectors from the origin to the two masses, and $G$ is the gravitational potential function implicitly provided by GEOGRAV. Note that we are therefore implicitly assuming that no energy is stored in the tension of the rod connecting the two masses.

Let's expand each of these terms. The translational kinetic energy is simply

$$KE_{trans} = \frac{1}{2} (m_A + m_B) |\dot{\vec r}^{ECI}|^2$$

The rotational kinetic energy is encoded in the rate of change of the quaternion:

$$KE_{rot} = \frac{1}{2} I |\vec \omega|^2 = \frac{m_B^2 + m_A^2}{m_A+m_B} d^2 |\dot q \circ \overline q|^2$$

Finally, $\vec r_A^{ECI}$ and $\vec r_B^{ECI}$ can be computed via $\vec r$ and $q$ as follows:
$$\vec d^B = (d, 0, 0) \ \ \ \ \ \ \ \ \vec d^{ECI} = q \circ \vec d^{B} \circ \overline q$$
$$\vec r_A^{ECI} = \vec r^{ECI} - \frac{m_B}{m_A + m_B} \vec d^{ECI}$$
$$\vec r_B^{ECI} = \vec r^{ECI} - \frac{m_B}{m_A + m_B} \vec d^{ECI}$$

To produce the E-L equations, we will codify the above via SymPy and let it do the work. Our state is given by $x = [\vec r \ \dot{\vec r} \ q \ \dot q]$, a total of 14 components.


In [1]:
import sympy as sy
from sympy.algebras.quaternion import Quaternion
from sympy.physics.vector import dynamicsymbols

In [2]:
q0,q1,q2,q3 = dynamicsymbols("q0 q1 q2 q3")
q0d,q1d,q2d,q3d = dynamicsymbols("q0 q1 q2 q3", 1)
q = Quaternion(q0, q1, q2, q3)
qd = Quaternion(q0d, q1d, q2d, q3d)
qc = q.inverse()

r1,r2,r3 = dynamicsymbols("r1 r2 r3")
r1d,r2d,r3d = dynamicsymbols("r1 r2 r3", 1)

mA,mB,d = sy.symbols("m_A m_B d")
m = mA + mB

w = (qd * qc).norm()
rKE = (1/2) * (mA**2 + mB**2) / m * d**2 * w**2
tKE = m * (r1d**2 + r2d**2 + r3d**2) / 2

rKE = sy.simplify(rKE)
rKE = rKE.collect([mA, mB]).subs({q0**2+q1**2+q2**2+q3**2:1})
T = rKE + tKE

dQ = Quaternion(0, d, 0, 0)
rQ = Quaternion(0, r1, r2, r3)
dECI = q * dQ * qc
dECI = dECI.subs({q0**2+q1**2+q2**2+q3**2:1})
rA = rQ - mB / m * dECI
rB = rQ + mA / m * dECI

G = sy.Function("G")
V = - (mA * G(rA.b, rA.c, rA.d) + mB * G(rB.b, rB.c, rB.d))

L = T - V
L

0.5*d**2*(m_A**2*(Derivative(q0(t), t)**2 + Derivative(q1(t), t)**2 + Derivative(q2(t), t)**2 + Derivative(q3(t), t)**2) + m_B**2*(Derivative(q0(t), t)**2 + Derivative(q1(t), t)**2 + Derivative(q2(t), t)**2 + Derivative(q3(t), t)**2))/(m_A + m_B) + m_A*G(-m_B*(d*q0(t)**2 + d*q1(t)**2 - d*q2(t)**2 - d*q3(t)**2)/(m_A + m_B) + r1(t), -m_B*(2*d*q0(t)*q3(t) + 2*d*q1(t)*q2(t))/(m_A + m_B) + r2(t), -m_B*(-2*d*q0(t)*q2(t) + 2*d*q1(t)*q3(t))/(m_A + m_B) + r3(t)) + m_B*G(m_A*(d*q0(t)**2 + d*q1(t)**2 - d*q2(t)**2 - d*q3(t)**2)/(m_A + m_B) + r1(t), m_A*(2*d*q0(t)*q3(t) + 2*d*q1(t)*q2(t))/(m_A + m_B) + r2(t), m_A*(-2*d*q0(t)*q2(t) + 2*d*q1(t)*q3(t))/(m_A + m_B) + r3(t)) + (m_A + m_B)*(Derivative(r1(t), t)**2 + Derivative(r2(t), t)**2 + Derivative(r3(t), t)**2)/2

Now let's produce an E-L equation for each independent coordinate.

In [3]:
from sympy.physics.mechanics import LagrangesMethod

LM = LagrangesMethod(L, [r1,r2,r3,q0,q1,q2,q3])
eqs = LM.form_lagranges_equations()
for eq in eqs:
    eq = sy.simplify(eq)

In [4]:
# Remove a bunch of ugly derivatives from the expression

a,b,c,x = map(sy.Wild, "abcx")
gx = sy.Function("g_x")
gy = sy.Function("g_y")
gz = sy.Function("g_z")

simp_eqs = []
for eq in eqs:
    eq = eq.replace(sy.Subs(sy.Derivative(G(x,b,c),x), x, a), gx(a,b,c))
    eq = eq.replace(sy.Subs(sy.Derivative(G(a,x,c),x), x, b), gy(a,b,c))
    eq = eq.replace(sy.Subs(sy.Derivative(G(a,b,x),x), x, c), gz(a,b,c))
    eq = eq.replace(mA + mB, sy.Symbol("m_T"))
    eq = sy.simplify(eq)
    simp_eqs.append(eq)
eqs = simp_eqs

In [5]:
# Collect argument expressions within the gravity calls

grav_calls = set()
for eq in eqs:
    grav_calls.update(eq.find(gx(a,b,c)))
    grav_calls.update(eq.find(gy(a,b,c)))
    grav_calls.update(eq.find(gz(a,b,c)))

args = set()
for call in grav_calls:
    for arg in call.args:
        args.add(sy.simplify(arg))

args = list(args)
args.sort(key=sy.srepr)
args = dict(zip([sy.Symbol(f"c_{i}") for i in range(len(args))], args))

simp_eqs = []
for eq in eqs:
    for k,v in args.items():
        eq = eq.replace(v, k)
    simp_eqs.append(eq)

# Replace all unique gravity calls with symbols
simplegrav_calls = set()
for eq in simp_eqs:
    simplegrav_calls.update(eq.find(gx(a,b,c)))
    simplegrav_calls.update(eq.find(gy(a,b,c)))
    simplegrav_calls.update(eq.find(gz(a,b,c)))

simplegrav_calls = list(simplegrav_calls)
simplegrav_calls.sort(key=sy.srepr)
calls = dict(zip([sy.Symbol(f"g_{i}") for i in range(len(simplegrav_calls))], simplegrav_calls))

simp_eqs2 = []
for eq in simp_eqs:
    for k,v in calls.items():
        eq = eq.replace(v, k)
    simp_eqs2.append(eq)

In [6]:
# Print expressions so that we can move them to C++
for k,v in args.items():
    print(k, ": ", v)
print()
for k,v in calls.items():
    print(k, ": ", v)

c_0 :  (-2*d*m_A*(q0(t)*q2(t) - q1(t)*q3(t)) + m_T*r3(t))/m_T
c_1 :  (-2*d*m_B*(q0(t)*q3(t) + q1(t)*q2(t)) + m_T*r2(t))/m_T
c_2 :  (-d*m_B*(q0(t)**2 + q1(t)**2 - q2(t)**2 - q3(t)**2) + m_T*r1(t))/m_T
c_3 :  (2*d*m_A*(q0(t)*q3(t) + q1(t)*q2(t)) + m_T*r2(t))/m_T
c_4 :  (2*d*m_B*(q0(t)*q2(t) - q1(t)*q3(t)) + m_T*r3(t))/m_T
c_5 :  (d*m_A*(q0(t)**2 + q1(t)**2 - q2(t)**2 - q3(t)**2) + m_T*r1(t))/m_T

g_0 :  g_x(c_2, c_1, c_4)
g_1 :  g_x(c_5, c_3, c_0)
g_2 :  g_y(c_2, c_1, c_4)
g_3 :  g_y(c_5, c_3, c_0)
g_4 :  g_z(c_2, c_1, c_4)
g_5 :  g_z(c_5, c_3, c_0)


In [7]:
x = [None]*7
x[0],x[1],x[2] = dynamicsymbols("r1 r2 r3", 2)
x[3],x[4],x[5],x[6] = dynamicsymbols("q0 q1 q2 q3", 2)

In [8]:
# Solve for the second derivative

solved_eqs = [sy.solve(simp_eqs2[i], x[i])[0] for i in range(len(eqs))]

Finally, we've computed $\frac{d}{dt} x$ as 

In [9]:
solved_eqs

[(g_0*m_A + g_1*m_B)/m_T,
 (g_2*m_A + g_3*m_B)/m_T,
 (g_4*m_A + g_5*m_B)/m_T,
 2.0*m_A*m_B*(-g_0*q0(t) + g_1*q0(t) - g_2*q3(t) + g_3*q3(t) + g_4*q2(t) - g_5*q2(t))/(d*(m_A**2 + m_B**2)),
 2.0*m_A*m_B*(-g_0*q1(t) + g_1*q1(t) - g_2*q2(t) + g_3*q2(t) - g_4*q3(t) + g_5*q3(t))/(d*(m_A**2 + m_B**2)),
 2.0*m_A*m_B*(g_0*q2(t) - g_1*q2(t) - g_2*q1(t) + g_3*q1(t) + g_4*q0(t) - g_5*q0(t))/(d*(m_A**2 + m_B**2)),
 2.0*m_A*m_B*(g_0*q3(t) - g_1*q3(t) - g_2*q0(t) + g_3*q0(t) - g_4*q1(t) + g_5*q1(t))/(d*(m_A**2 + m_B**2))]